In [ ]:
! curl -L https://git.io/GeoLite2-City.mmdb -o GeoLite2-City.mmdb 
! pip3 install geoip2
! pip3 install nslookup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   152  100   152    0     0    294      0 --:--:-- --:--:-- --:--:--   294
100   626  100   626    0     0   1011      0 --:--:-- --:--:-- --:--:--  1011
100 68.4M  100 68.4M    0     0  42.2M      0  0:00:01  0:00:01 --:--:-- 95.6M
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPREC

In [16]:
import folium
from folium.plugins import MarkerCluster
import geoip2.database
import io
from nslookup import Nslookup
import os
import pandas as pd
import requests
import sys
import warnings
import zipfile


f = open(os.devnull, 'w')
sys.stderr = f
warnings.filterwarnings("ignore")

In [17]:
r = requests.get("http://s3-us-west-1.amazonaws.com/umbrella-static/top-1m.csv.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [18]:
df = pd.read_csv ('top-1m.csv', names=['Rank', 'DNS'], header=None)
df['DNS'] = df['DNS'].astype('str')

In [19]:
dns_query = Nslookup(dns_servers=["8.8.8.8"])
topuk = df[df["DNS"].str.contains(".co.uk")]

In [20]:
topuk['ip_address'] = topuk['DNS'].map(lambda host:dns_query.dns_lookup(host).answer)
topuk["ip_address"] = topuk["ip_address"].str[0]
topuk = topuk.dropna()
topuk

,Rank,DNS,ip_address
2101,2102,google.co.uk,142.250.72.67
2463,2464,amazon.co.uk,54.239.33.58
2650,2651,www.google.co.uk,142.250.69.227
3751,3752,adservice.google.co.uk,142.250.72.34
4102,4103,bbci.co.uk,212.58.249.207
...,...,...,...
21701,21702,cmp.dmgmediaprivacy.co.uk,184.28.48.143
21702,21703,dmgmediaprivacy.co.uk,184.28.48.143
21842,21843,api.official-deals.co.uk,54.239.174.113
21875,21876,nsi.activity.api.bbc.co.uk,52.211.38.76


In [21]:
def get_latitude(ip):
    try:
        reader = geoip2.database.Reader('GeoLite2-City.mmdb')
        response = reader.city(ip)
        output = response.location.latitude
        return output
    except:
        return pd.np.nan


def get_longitude(ip):
    try:
        reader = geoip2.database.Reader('GeoLite2-City.mmdb')
        response = reader.city(ip)
        output = response.location.longitude
        return output
    except:
        return pd.np.nan

def get_country(ip):
    try:
        reader = geoip2.database.Reader('GeoLite2-City.mmdb')
        response = reader.city(ip)
        output = str(response.country.iso_code)
        return output
    except:
        return pd.np.nan

unique_ips = topuk['ip_address'].unique()
unique_ips = pd.Series(unique_ips, index = unique_ips)
topuk['Latitude'] = topuk['ip_address'].map(unique_ips.apply(get_latitude))
topuk['Longitude'] = topuk['ip_address'].map(unique_ips.apply(get_longitude))
topuk['Country'] = topuk['ip_address'].map(unique_ips.apply(get_country))



In [22]:
topuk = topuk.dropna()
topuk

,Rank,DNS,ip_address,Latitude,Longitude,Country
2101,2102,google.co.uk,142.250.72.67,37.7510,-97.8220,US
2463,2464,amazon.co.uk,54.239.33.58,53.3382,-6.2591,IE
2650,2651,www.google.co.uk,142.250.69.227,37.7510,-97.8220,US
3751,3752,adservice.google.co.uk,142.250.72.34,37.7510,-97.8220,US
4102,4103,bbci.co.uk,212.58.249.207,51.4964,-0.1224,GB
...,...,...,...,...,...,...
21701,21702,cmp.dmgmediaprivacy.co.uk,184.28.48.143,32.7767,-96.8050,US
21702,21703,dmgmediaprivacy.co.uk,184.28.48.143,32.7767,-96.8050,US
21842,21843,api.official-deals.co.uk,54.239.174.113,37.7510,-97.8220,US
21875,21876,nsi.activity.api.bbc.co.uk,52.211.38.76,53.3382,-6.2591,IE


In [38]:
import folium


map = folium.Map(location=[50, 0], tiles='cartodb positron', zoom_start=4)

mcluster = MarkerCluster().add_to(map)

for index, row in topuk.iterrows():
    folium.Marker(location=[row['Latitude'],row['Longitude']], popup=row.DNS).add_to(mcluster)

map